In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
from Feat_eng import (Y_train, X_train, Y_test, X_test, 
    c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, 
    Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().dro

In [18]:
# Randomized parameter grid search on all features

rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": np.arange(10, 20, 1), 
          "max_samples": [.7,.8,.9]}
rf_random = RandomizedSearchCV(rf, params, n_iter=25, cv=5, n_jobs=-1)

rf_random.fit(X_train,Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'max_samples': [0.7, 0.8, 0.9],
                                        'min_samples_leaf': [6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])})

In [22]:
# Run a PCA on all data:
scaler_pca = StandardScaler().fit(X_train)
features = scaler_pca.transform(X_train)
df1_scal = pd.DataFrame(features, columns = X_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df1_scal)

df_reduced = pd.DataFrame(pca.components_, columns = X_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat.append(ff)

feat.append('log_value')
X_PCA_train = X_train[feat]
X_PCA_test = X_test[feat]


In [37]:
# Randomized parameter grid search for PCA on all data:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf2_random = RandomizedSearchCV(rf2, params, n_iter=50, cv=5, n_jobs=-1)

rf2_random.fit(X_PCA_train,Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [19]:
# Results of Random Forest on all features and data
print(rf_random.best_params_)
print(rf_random.best_score_)
Y_pred = rf_random.predict(X_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_train)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', RMSE)
Y_pred = rf_random.predict(X_test)
pred_unlog = np.exp(Y_pred)s
Y_unlog = np.exp(Y_test)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', RMSE)

{'n_estimators': 17, 'min_samples_split': 16, 'min_samples_leaf': 8, 'max_samples': 0.8}
0.9789603417142635
The training RMSE is: 54.24057055339063
The test RMSE is: 73.83045393397678


In [38]:
# Results of Random Forest on all features with PCA
print(rf2_random.best_params_)
print(rf2_random.best_score_)
Y_pred = rf2_random.predict(X_PCA_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_train)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', RMSE)
Y_pred = rf2_random.predict(X_PCA_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_test)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', RMSE)

{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_samples': 0.4}
0.9795886800171036
The training RMSE is: 58.338159482642055
The test RMSE is: 74.03181721013713


## Cluster 1 Random Forest

In [39]:
# Randomized parameter grid search for Cluster 1:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf1= RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf1.fit(c1_train,Y_c1train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [41]:
# Results of Random Forest on Cluster 1:
print(rf1.best_params_)
print(rf1.best_score_)
Y_pred = rf1.predict(c1_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1train)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', RMSE)
Y_pred = rf1.predict(c1_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1test)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', RMSE)

{'n_estimators': 200, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_samples': 0.7}
0.9589013092680627
The training RMSE is: 62.59074835444834
The test RMSE is: 77.29867665721176


## Cluster 2 Random Forest

In [42]:
# Randomized parameter grid search for Cluster 2:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf2 = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf2.fit(c2_train,Y_c2train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [43]:
# Results of Random Forest on Cluster 2:
print(rf2.best_params_)
print(rf2.best_score_)
Y_pred = rf2.predict(c2_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2train)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', RMSE)
Y_pred = rf2.predict(c2_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2test)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', RMSE)

{'n_estimators': 150, 'min_samples_split': 14, 'min_samples_leaf': 4, 'max_samples': 0.6}
0.9760609762069367
The training RMSE is: 48.78416964441518
The test RMSE is: 71.28870570176925


## Cluster 3 Random Forest

In [44]:
# Randomized parameter grid search for Cluster 3:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf3 = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf3.fit(c3_train,Y_c3train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [45]:
# Results of Random Forest on Cluster 3:
print(rf3.best_params_)
print(rf3.best_score_)
Y_pred = rf3.predict(c3_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3train)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', RMSE)
Y_pred = rf3.predict(c3_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3test)
RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', RMSE)

{'n_estimators': 100, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_samples': 0.8}
0.9644713207194533
The training RMSE is: 43.5443699658256
The test RMSE is: 83.40997365647577
